In [ ]:
if True:
    !pip install yt_dlp
    #!pip uninstall torch
    # os.system('c: && cd ProgramData/Anaconda3 && python -m pip install --force-reinstall torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu117')
    #!pip uninstall torchvision
    #!pip uninstall torchaudio
    !pip install --force-reinstall torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu117
    !pip install ftfy regex tqdm
    !pip install https://github.com/aboSamoor/pycld2/zipball/e3ac86ed4d4902e912691c1531d0c5645382a726
    !pip install git+https://github.com/openai/CLIP.git
    !pip install transformers
    !pip install sacremoses
    !pip install textblob
    !pip install pickle
    !pip install matplotlib
    

import yt_dlp
import matplotlib
import torch as th
import cv2
import os
import pickle
import pycld2
import clip
from PIL import Image
from sklearn import svm
import numpy as np




In [2]:
############################################################################################### The point of the following code is to get a bunch of links to clean Minecraft videos. get the initial links from YT search and manual selection. account for whats already been seen before adding something.

In [3]:
all_links_seen = []
spam_links=[]
ham_links=[]
downloaded_videos=[]

In [4]:
# check which links are already  HAM and SPAM classified
# get all downloaded video (720 20 hz 10 mins)

with open('DATASET/all/ham_video.txt','r') as ham_videos:
    ham_videos = ham_videos.read().split('\n')
with open('DATASET/all/ham_transcript.txt','r') as ham_transcript:
    ham_transcript = ham_transcript.read().split('\n')
with open('DATASET/all/ham_metadata.txt','r') as ham_metadata:
    ham_metadata = ham_metadata.read().split('\n')

ham_links = list(set(ham_videos).union(set(ham_transcript)))
for link in ham_links:
    if not link in all_links_seen:
        all_links_seen.append(link)


# gt list of already dientifited spam
with open('DATASET/all/spam_video.txt', 'r') as file:
    spam_video = file.read().split('\n')
with open('DATASET/all/spam_transcript.txt', 'r') as file:
    spam_transcript = file.read().split('\n')
with open('DATASET/all/spam_metadata.txt','r') as spam_metadata:
    spam_metadata = spam_metadata.read().split('\n')

spam_links = list(set(spam_transcript).union(set(spam_video)).union(set(spam_metadata)))
for link in spam_links:
    if not link in all_links_seen:
        all_links_seen.append(link)

In [5]:
# MANUAL LINKS IMPORT
# get the list of manually added links to check, get video data, add to video list
with open('spamham_samples/manual_clean_ids.txt', 'r') as file:
    manual_links = file.read().split('\n')

# add manually selected links to list of links to check for download
# if this manual link to add was already added by all seen links, done add it again
for link in manual_links:
    if not link in all_links_seen:
        all_links_seen.append(link)


"""# LOAD VIDEO DETAILS FROM FILE
import pickle
#pickle.dump(all_videos_to_search, open('manual_metad','wb'))
all_videos_to_search = pickle.load(open('manual_metad','rb'))"""

"# LOAD VIDEO DETAILS FROM FILE\nimport pickle\n#pickle.dump(all_videos_to_search, open('manual_metad','wb'))\nall_videos_to_search = pickle.load(open('manual_metad','rb'))"

In [6]:

# DEFINE SEARCH & REGEX TERMS
# Search terms
search_queries = [
    "minecraft tutorial survival vanilla new world lets play",
    "minecraft tutorial hardcore new world lets play 1.16",
    "minecraft how to survival vanilla new world lets play 1.16",
    "minecraft tutorial first start survival lets play 1.16",
    "minecraft lets play tutorial survival new world 1.16",
    "minecraft lets play tutorial hardcore new world",
    "minecraft guide walkthrough survival new world 1.16",
    "minecraft how to first night playthrough",
    "minecraft tutorial lets play no webcam 1.16",
    "minecraft survival hardcore tutorial how to guide learn basics beginner lets play walkthrough gameplay new world first night first day 1.16",
    "minecraft survival vanilla guide let's play",
    "minecraft vanilla survival let's play",
    "minecraft vanilla hardcore let's play",
    "minecraft early game lets play vanilla"
]

# Title/tags
exclusion_terms = [
    'mods',
    'modded',
    'modding',
    '-mod',
    'ps3',
    'ps4',
    'ps5',
    'xbox',
    'bedrock',
    'pocket',
    'bed rock',
    'no commentary',
    'creative',
    'multiplayer',
    'server',
    'playstation',
    'timelapse',
    'animation',
    'minecraft pe',
    'pocket edition',
    'bedrock edition',
    'skyblock',
    'realistic',
    'shader',
    'how to install',
    'how to download',
    'realmcraft',
    'realms',
    'animation',
    'animatic',
    'music video',
    'movie',
    'friend',
    'parody',
    'island',
    'skyblock',
    'squad',
    'hack',
    'hacked',
    'hacking',
    'battle',
    'minecraft but',
    'minecraft, but',

    ' we ',  # dont want multiplayer
    ' smp ',
    'rtx',
    'feature',
    'lucky block',
    'bedrock',
    'pocket',
    ' pe ',
    'unturned',
    'roblox',
    'dimension',
    ' top ',

    'broken', #dont want hacks or glitches
    'broke',
    'glitch',
    'texturepack',
    'texture pack',
    'resourcepack',
    'resource pack',
    

    ' end', # dont want late game
    'nether',
    'dimension',
    'dragon',
    'wither',

    'no commentary', # filters to ensure useful langauge input
    'silent',
    'asmr',
    'no talking',
    'silent',
    'silence'
]

title_inclusion_terms = [
    ['minecraft'],
#    ['surviv', 'hardcore'],
#    ['guide', 'tutorial', 'how to', 'surviv', 'learn', 'basic','noob','dummies'],
    ["let's play", "lets play",'walkthrough','playthrough', 'walk through', 'play through', 'gameplay'],
#    ['first','begin','start','fresh','new'],
    ['world','game','play','day','night']
] 


In [7]:
# SEARCH YOUTUBE WITH SEACH TERMS AND GET RESULTS
# just collect lots of potential videos, whittle down in next step
if False:


    YDL_OPTIONS = {'noplaylist':True, 'download':False, 'ignoreerrors':True, 'list_subs':True, 'external-downloader':'aria2c', 'extract_flat':True, }

    with yt_dlp.YoutubeDL(YDL_OPTIONS) as ydl:
        for search_query in search_queries:
            # get top search results for query, add results to top_MC_videos[]
            search_results = ydl.extract_info(f"ytsearchall:{search_query}", download=False)
            for result in search_results['entries']:
                link = result['id']
                if not (link in all_links_seen):
                    all_links_seen.append(link)

In [8]:
# SNOWBALL FROM FOUND HAM VIDEOS
if False:
    ## ONLY DO THIS AFTER INITIAL SWEEP ON SEARCHES AND ALL ARE CLASSIFIED
    MC_channels = []

    # get channels from ham links found so far
    with yt_dlp.YoutubeDL(YDL_OPTIONS) as ydl:
        for link in ham_links:
            video = ydl.extract_info(link, download=False)
            channel = video['channel']
            if not (channel in MC_channels):
                MC_channels.append(channel)

    # get all videos from ham channels
    with yt_dlp.YoutubeDL(YDL_OPTIONS) as ydl:
        # get videos from collected channels, check if minecraft related and save
        for channel in MC_channels:
            channel_videos = ydl.extract_info(f'https://www.youtube.com/channel/{channel}/videos')
            for video in channel_videos:
                if not (video['id'] in ham_links) and not (video['id'] in spam_links):
                    all_links_seen.append(video)

In [9]:
# UPDATE TEXT FILE OF ALL LINKS SEEN
with open('DATASET/all/all_seen_links.txt', 'r') as file:
    all_seen_links_file = file.read().split('\n')
    for link in all_seen_links_file:
        if not link in all_links_seen:
            all_links_seen.append(link)

while '' in all_links_seen:
    all_links_seen.remove('')

with open('DATASET/all/all_seen_links.txt', 'w') as file:
    for link in all_links_seen:
        file.write(link+'\n')

############################################################################################################################################ finish gettings links worth checking. write link to file


In [10]:
len(all_links_seen)

5907

In [11]:
### LOAD FRAME SPAM/HAM CALSSIFIER
# load CLIP
device = "cuda" if th.cuda.is_available() else "cpu"
CLIP, preprocess = clip.load("RN50x64", device=device)

# LOAD SVM
svm = pickle.load(open('SVM4', 'rb'))

def frames_to_class(frames):
    with th.no_grad():
        # pass images as torch array to CLIP image encoder
        image_features = CLIP.encode_image(frames).cpu().numpy()

        # pass CLIP image features to SVM to predict label
        estimations = svm.predict(image_features).astype(np.int8)
        label = np.median(estimations, axis=0)
        return int(label)


### FOR CHECKING VIDEO SPAM HAM.

import time

def elapsedTime(start):
    now = time.time()
    print('['+str(now-start)+']')
    return now

# https://pypi.org/project/yt-dlp/#usage-and-options
def longer_than_10_mins(info, *, incomplete):
    """Download only videos longer than 10 minutes"""
    duration = info.get('duration')
    if duration and duration < 600:
        return 'The video is too short'
ydl_opts={'match_filter':longer_than_10_mins, 'download':False, 'fps':60}

100%|█████████████████████████████████████| 1.26G/1.26G [03:37<00:00, 6.23MiB/s]


In [12]:
# RE-CHECK SPAM HAM IN FILES
with open('DATASET/all/ham_video.txt','r') as ham_videos:
    ham_videos = ham_videos.read().split('\n')
with open('DATASET/all/ham_transcript.txt','r') as ham_transcript:
    ham_transcript = ham_transcript.read().split('\n')
with open('DATASET/all/ham_metadata.txt','r') as ham_metadata:
    ham_metadata = ham_metadata.read().split('\n')

ham_links = list(set(ham_videos).intersection(set(ham_transcript)).intersection(set(ham_metadata)))



# gt list of already dientifited spam
with open('DATASET/all/spam_video.txt', 'r') as file:
    spam_video = file.read().split('\n')
with open('DATASET/all/spam_transcript.txt', 'r') as file:
    spam_transcript = file.read().split('\n')
with open('DATASET/all/spam_metadata.txt','r') as spam_metadata:
    spam_metadata = spam_metadata.read().split('\n')

spam_links = list(set(spam_transcript).union(set(spam_video)).union(set(spam_metadata)))


In [13]:
############################################################################################################################################  from seen links that have not been classified, one at a time, get yt_search metadata for it online, check it for spam/ham, classify, and send to appropriate file.

# from seen links and spam/ham so far, get list of links that need to be classified
import random
links_to_check = list(   set(all_links_seen) - set(spam_links).union(set(ham_links))   ) 
random.shuffle(links_to_check)


len(links_to_check)

4044

In [14]:
# CHECK HAM/SPAM ON EMTADATA, THEN VIDEO, THEN AUDIO


# load target LM's tokenizer for checking WPM as tokens/ms  --- note: not implemented in the end
import pycld2 as cld2
from transformers import TransfoXLTokenizer
import re
tokenizer = TransfoXLTokenizer.from_pretrained("transfo-xl-wt103")
YDL_OPTIONS = {'noplaylist':True, 'download':False, 'ignoreerrors':True, 'externaldownloader':'aria2c', 'quiet':True, 'downloader':'aria2c' } #'list_subs':True,


for link in links_to_check: # do not check links which have already had vidfeo approved. ones disapproved dont reach this stage.


    is_spam=False
    if (link in ham_videos) or (link in spam_links): # dont re-check already classified videos.
        continue



    ## from link, fetch metaadata
    with yt_dlp.YoutubeDL(YDL_OPTIONS) as ydl:
        video = ydl.extract_info(link, download=False)

    if video == None:
        print("VIDEO NONE NULL")
        continue








    ################## CHECK LINK METADATA FOR SPAM/HAM (title, tags, transcript...)
    if not is_spam:

        # check video duration
        if video['duration'] < 600:
            is_spam=True
            print('durationspam=')

            
        # CHECK TITLE AGAINST REGEX
        title = video['title']
        for term in exclusion_terms:
            if term.lower() in title.lower():
                is_spam = True
                print(term.lower(), 'titlespamex=')
        for term in title_inclusion_terms:
            includes_subterm=False
            for subterm in term:
                if subterm.lower() in title.lower():
                    includes_subterm = True
            if not includes_subterm:
                is_spam=True
                print(term, 'titlespaminc=')
        # Check for multiplayer in title [... with H..] e.g. "playing minecraft with Harry"
        if 'with' in title:
            char_after_with = title[title.lower().find('with') + 5]
            if char_after_with.lower() != char_after_with:
                is_spam = True
                print(title, 'titlespam: "with"')
        #check for episode number in title and make sure it is small
        nums = re.findall(r'\d+', title) #https://stackoverflow.com/questions/17007257/python-3-finding-the-last-number-in-a-string
        if len(nums)>0:
            if int(nums[len(nums)-1]) > 30:
                is_spam=True
                print('episode spam NUMBER,', title)    
        
        
        # CHECK DESCRIPTION AGAINST REGEX
        # cut description or referring to a buch of different youtube videos
        description = video['description'].lower()

        if 'youtube.' in description:
            description = description.split('youtube.')[0]
        if '//youtu.be' in description:
            description = description.split('//youtu.be')[0]

        for term in exclusion_terms:
            if term in [' we ']: # these words are check for in title but not description becuase description is too likely to have them while being good
                continue
            if term in description and (not (term=='creative' and ('survival' in description or 'hardcore' in description))) and ( not (term=='server' and ('discord server' in description) )):
                is_spam=True
                print(term, 'descriptionspam')

                
        # CHECK TAGS AGAINST SPAM TAGS (remove ps3/xbox/console/bedrock/ps4/ps5/xbox series...
        tags = video['tags']
        for term in exclusion_terms:
            if term in tags:
                is_spam=True
                print(term, 'tagspam=')

                
        # CHECK LANGUAGE (title, description)
        _,_,details=cld2.detect(description)
        if details[0][0]!= 'ENGLISH':         # get only English videos
            is_spam=True
            print('description spam language',details[0])
        _,_,details=cld2.detect(title)
        if details[0][0]!= 'ENGLISH':         # get only English videos
            is_spam=True
            print('lang spam title', details[0])
    if is_spam:
        print('!!!!!!!!!!!METADATASPAM')
        with open('DATASET/all/spam_metadata.txt', 'a') as file:
            file.write(link+'\n')

    else:
        print('!!!!!!METADATA HAM')
        with open('DATASET/all/ham_metadata.txt', 'a') as file:
            file.write(link+'\n')





















        
    
    ######################## CHECK VIDEO SPAM/HAM
    ##############
    
    if (not is_spam) and not (link in ham_videos):
        formats = video['formats']
        duration = video['duration']

        # get video @ 3Hz    #https://stackoverflow.com/questions/66272740/extract-specific-frames-of-youtube-video-without-downloading-video
        print("downloading new video...")
        #fps = format['fps']
        #print(fps)
        #url = format['url']
        #print('got format',link,url)
        #num_frames = (fps*duration) #cut off last second due to possible variation in fps.
        
        # pre-download whole video
        videos_folder = 'DATASET/all/videos/'
        reduced_videos_folder = 'DATASET/all/clean_videos/'
        output_filename = videos_folder+link
        
        try:
            ydl_dwnld_opts={'match_filter':longer_than_10_mins, 
                            'download':True, 
                            'outtmpl':output_filename, 
                            "format":"(bestvideo[height=720][fps=60]/bestvideo)+bestaudio/best",
                            'downloader':'aria2c',
                            'external_downloader':'aria2c',
                            'externaldownloader':'aria2c',
                            }
            with yt_dlp.YoutubeDL(ydl_dwnld_opts) as ydl:
                full_link = 'https://youtu.be/'+link
                ydl.download(full_link)

        except:
            ydl_dwnld_opts={'match_filter':longer_than_10_mins, 
                            'download':True, 
                            'outtmpl':output_filename, 
                            "format":"(bestvideo[height=720]/bestvideo)+bestaudio/best",
                            'downloader':'aria2c',
                            'external_downloader':'aria2c',
                            'externaldownloader':'aria2c',
                            }
            with yt_dlp.YoutubeDL(ydl_dwnld_opts) as ydl:
                full_link = 'https://youtu.be/'+link
                ydl.download(full_link)

        finally:
            ydl_dwnld_opts={'match_filter':longer_than_10_mins, 
                            'download':True, 
                            'outtmpl':output_filename,
                            "format":"(bestvideo[height=480]/bestvideo)+bestaudio/best",
                            'downloader':'aria2c',
                            'external_downloader':'aria2c',
                            'externaldownloader':'aria2c',
                            }
            with yt_dlp.YoutubeDL(ydl_dwnld_opts) as ydl:
                full_link = 'https://youtu.be/'+link
                ydl.download(full_link)

        filenames = os.listdir(videos_folder)
        for filename in filenames:
            if filename.split('.')[0] == link:
                output_filename = videos_folder+filename
                break
        cap = cv2.VideoCapture(output_filename)
        fps = cap.get(cv2.CAP_PROP_FPS)

        
        ###
        #frame_index=fps*1*60 # skip into start of video. REMEMBER DURING ACTUAL VIDEO EXTRACTION TO CUT TO ACTUAL CLEAN GAMEPLAY # go at least 2 minutes into video. if still mostly spam, stop classifying - it is probably spam
        #cap.set(1, round(frame_index))
        frame_index=0 # skip into start of video. REMEMBER DURING ACTUAL VIDEO EXTRACTION TO CUT TO ACTUAL CLEAN GAMEPLAY # go at least 2 minutes into video. if still mostly spam, stop classifying - it is probably spam
        ###

        is_spam=False
        frames=[]
        spamham_labels=[]
        # get next frame
        
        print('|',end='')
        ret=True
        while ret and (not is_spam):
            now = time.time()

            #get frame
            ret, frame = cap.read()
            if not ret:
                print('NOT RET')
                break
            #convert
            frame = Image.fromarray(frame)

            with th.no_grad():
                frame = preprocess(frame).unsqueeze(0).to(device) # preprocess each frame as they come in so we dont reprocess each frame each new window slide
            # add to list of current frames (we are doing sliding widow of size 7 across video at 3Hz)
            frames.append(frame)
            

            ##### NOT ENOUGH COMPUTE: we will check eery 10 seconds of video for spam/ham. damaging types of spam ham are consistent: creative, multiplayer, sever, mods, texturepacks, shaders. skyblock, console version, unrelated, nether, etc... probably fine to check this inrequently
            # skip enough frames to get 3Hz
            frame_index += fps*10
            cap.set(cv2.CAP_PROP_POS_FRAMES, round(frame_index))
            # keep frames[] as 7-wide window across episode
            if len(frames) == 8:
                frames.pop(0)
                frames.pop(0)

            

            # use median filter of kernel_size=7 across vidoe to classify clean frames, as described in paper
            if len(frames) == 7:
                classification_window_frames = th.cat(frames)
                label = frames_to_class(classification_window_frames)
                print(label,end='')
                spamham_labels.append(label)

                if frame_index/fps >= 120: # dont make jusdgement on if video is spam/ham until 2 mins in. if mostly spam, the video is marked as spam           # REMMEBER: up to 1 minute of video may now be dirty. when loading dataset use the casifier and crop these areas of video to save on resources
                    # if the median of all frames is

                    num_clean = np.count_nonzero(np.asarray(spamham_labels)==1)
                    total = len(spamham_labels)
                    clean_fraction = float(num_clean)/float(total)
                    #print('{'+str(clean_fraction)+'}', end='')
                    if (clean_fraction) < 0.7:
                        print('SPAM VIDEO!')
                        is_spam=True
                        with open('DATASET/all/spam_video.txt', 'a') as file:
                            file.write(link+'\n')
                        break

        
        # if stopped classifying frames, its because either because spam or ham
        cap.release()
        if not is_spam:
            print('VIDEO HAM!!!')
            # write that this video is ham video
            with open('DATASET/all/ham_video.txt', 'a') as file:
                file.write(link+'\n')
            command = "del "+output_filename
            # since the video is ham is shuold be stored permanently: it needs to be downscaled to 360p as in VPT and sampled at 20Hz
            reduced_output_filename = reduced_videos_folder+filename
            command = "ffmpeg -i "+output_filename+' -vf "fps=20,scale=640:360,format=yuv420p" vcodec h265 -preset medium -crf 30 '+output_filename+" && del "+output_filename
            os.system(command)























        ########################
        #########################  CHECK TRANSCRIPT SPAM/HAM DURING VIDEO

       
        # https://superuser.com/questions/927523/how-to-download-only-subtitles-of-videos-using-youtube-dl
        # check subtitles LANGAUGE and WPM
        if False: #(not is_spam) and not (link in ham_transcript):
            pass
            """
            # FIRST CHECK FOR EXISTING SUBTITLES:
            try write-sub --sub-lang en --skip-download URL 
             else:
              write-auto-sub --sub-lang en --skip-download URL 
            # check if subtitiles are available in English
                subtitles = video.get('subtitles')
            if not subtitles:
                subtitles = video.get('automatic_captions')
            # if subtitiles are available, check if english is available, if not, this video is not english
            if subtitles and not subtitles.get('en'):
                is_spam=True
            print(video['subtitles'], 'spam=',is_spam)
            
            # Check WPM. we dont want video with too high a WPM. LM can only fit so many words and we dont watn to use up all the context for a single minute of video. lower WPM also means we can use large D which lets LM see farther back in time. High WPM also may indicate multiplayer which is a very noisy langauge signal, especially since other person wont be seeing what is shown in the video, their langauge will be about something completely different.
            if subtitles:
                transcript = subtitles
                num_tokens = len(tokenizer(' '.join(transcript)))
                video_mins = video['duration']/60
                WPM = num_tokens/video_mins     # WARNING: variable is called WPM as simplification, its actually tokens per minute
                if WPM<180 or WPM>280:
                    is_spam = True
                    print('WPm spam')


            # if no subtitles exist, check subtitles with OpenAI Whisper tiny - we only need to measure wpm and check language
        
            if is_spam:
                with open('DATASET/all/spam_transcript.txt', 'a') as file:
                    file.write(video['id']+'\n')
            else:
                with open('DATASET/all/ham_transcript.txt','a') as file:
                    file.write(video['id']+'\n')


            ############ CHECK LANGUAGE HAM/SPAM
            ############

            ### LOAD AUDIO TRANSCRIBER
            #from stable_whisper import load_model
            #from stable_whisper import stabilize_timestamps
            #model = load_model('base')


            # want to download slowly to not get banned by google - dont use multiple workers
            for video in len(all_videos_to_search):
                is_spam=False
                link = video['id']
                formats = video['formats']
                duration = video['duration']

                # skip if video is already downloaded.
                if (link in ham_links) or (link in spam_links):
                    continue


                ############## CHECK LANGUAGE/TRANSCRIPT SPAM/HAM
                ##############

                # get audio sample & check spamham
                if not is_spam:
                    ydl.getaudio(link, 00:00-00:20, download=audiofile)
                    # check langauge, WPM
                    language = model.detectlanguage(audiofile)
                    if language != 'english':
                        is_spam=True
                        continue

                # if clean, save link to file
                with open('ham_language.txt','a') as clean_links_file:
                    clean_links_file.write(video['id'])

                # download video and scale to 128
                #os.system(dlp download video.link -ouput link.mp4)
                #ffmpeg()
            """




xbox titlespamex=
no commentary titlespamex=
no commentary titlespamex=
["let's play", 'lets play', 'walkthrough', 'playthrough', 'walk through', 'play through', 'gameplay'] titlespaminc=
['world', 'game', 'play', 'day', 'night'] titlespaminc=
episode spam NUMBER, 1 HOUR of Minecraft: Xbox 360 Edition (Survival Mode) - No Commentary
xbox descriptionspam
no commentary descriptionspam
friend descriptionspam
no commentary descriptionspam
xbox tagspam=
!!!!!!!!!!!METADATASPAM


         Install PhantomJS to workaround the issue. Please download it from https://phantomjs.org/download.html
         n = Q_GUSsHR1UGdVGD ; player = https://www.youtube.com/s/player/1cbba2b4/player_ias.vflset/en_US/base.js


durationspam=
bedrock titlespamex=
bedrock titlespamex=
["let's play", 'lets play', 'walkthrough', 'playthrough', 'walk through', 'play through', 'gameplay'] titlespaminc=
['world', 'game', 'play', 'day', 'night'] titlespaminc=
bedrock descriptionspam
bedrock descriptionspam
nether descriptionspam
bedrock tagspam=
bedrock tagspam=
nether tagspam=
!!!!!!!!!!!METADATASPAM


         Install PhantomJS to workaround the issue. Please download it from https://phantomjs.org/download.html
         n = M6Ua6sm2JDoqvUy ; player = https://www.youtube.com/s/player/1cbba2b4/player_ias.vflset/en_US/base.js


episode spam NUMBER, Item Transport Pipes! ▫ The Minecraft Survival Guide (Tutorial Lets Play) [Part 172]
realistic descriptionspam
!!!!!!!!!!!METADATASPAM


...

!!!!!!METADATA HAM
downloading new video...
[youtube] Extracting URL: https://youtu.be/OvxoNNASjDw
[youtube] OvxoNNASjDw: Downloading webpage
[youtube] OvxoNNASjDw: Downloading android player API JSON


         Install PhantomJS to workaround the issue. Please download it from https://phantomjs.org/download.html
         n = Qb-anOp0rYJ1xMk ; player = https://www.youtube.com/s/player/1cbba2b4/player_ias.vflset/en_US/base.js


[info] OvxoNNASjDw: Downloading 1 format(s): 302+251
[youtube] Extracting URL: https://youtu.be/OvxoNNASjDw
[youtube] OvxoNNASjDw: Downloading webpage
[youtube] OvxoNNASjDw: Downloading android player API JSON


         Install PhantomJS to workaround the issue. Please download it from https://phantomjs.org/download.html
         n = No2u1k-mmUDkSxb ; player = https://www.youtube.com/s/player/1cbba2b4/player_ias.vflset/en_US/base.js


[info] OvxoNNASjDw: Downloading 1 format(s): 302+251
[download] Destination: DATASET\all\videos\OvxoNNASjDw.f302.webm
[download]  55.2% of  268.61MiB at    6.04MiB/s ETA 00:19